In [1]:
import numpy as np
import tensorflow as tf
import time

import matplotlib.pyplot as plt
%matplotlib inline 

from gpflow.likelihoods import Gaussian
from gpflow.kernels import RBF, White
from gpflow.mean_functions import Constant
from gpflow.models.sgpr import SGPR, GPRFITC
from gpflow.models.svgp import SVGP
from gpflow.models.gpr import GPR
from gpflow.training import AdamOptimizer, ScipyOptimizer

from scipy.cluster.vq import kmeans2
from scipy.stats import norm
from scipy.special import logsumexp

from doubly_stochastic_dgp.dgp import DGP
from datasets import Datasets
datasets = Datasets(data_path='data/')

/home/gonzalo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = datasets.all_datasets['boston'].get_data()
X, Y, Xs, Ys, Y_std, Y_mean = [data[_] for _ in ['X', 'Y', 'Xs', 'Ys', 'Y_std', 'Y_mean']]
print('N: {}, D: {}, Ns: {}'.format(X.shape[0], X.shape[1], Xs.shape[0]))

N: 455, D: 13, Ns: 51


In [3]:
print("Is normalized?: {}".format(np.allclose(0,np.mean(X))))
print(np.std(X))
print(Y_std)

Is normalized?: True
1.0751266056104234
[7.79674467]


In [4]:
def make_dgp(X, Y, Z, L):
    D = X.shape[1]
    
    # the layer shapes are defined by the kernel dims, so here all hidden layers are D dimensional 
    kernels = []
    for l in range(L):
        kernels.append(RBF(D))
        
    # between layer noise (doesn't actually make much difference but we include it anyway)
    for kernel in kernels[:-1]:
        kernel += White(D, variance=1e-5) 
        
    mb = 1000 if X.shape[0] > 1000 else None 
    model = DGP(X, Y, Z, kernels, Gaussian(), num_samples=10, minibatch_size=mb)

    # start the inner layers almost deterministically 
    for layer in model.layers[:-1]:
        layer.q_sqrt = layer.q_sqrt.value * 1e-5
    
    return model

In [5]:
Z_100 = kmeans2(X, 100, minit='points')[0]
m_dgp2 = make_dgp(X, Y, Z_100, 2)

In [6]:
AdamOptimizer(0.01).minimize(m_dgp2, maxiter=500)

In [7]:
def batch_assess(model, X, Y):
    n_batches = max(int(X.shape[0]/1000.), 1)
    lik, sq_diff = [], []
    for X_batch, Y_batch in zip(np.array_split(X, n_batches), np.array_split(Y, n_batches)):
        l, sq = assess_sampled(model, X_batch, Y_batch)
        lik.append(l)
        sq_diff.append(sq)
    lik = np.concatenate(lik, 0)
    sq_diff = np.array(np.concatenate(sq_diff, 0), dtype=float)
    return np.average(lik), np.average(sq_diff)**0.5

S = 100
def assess_sampled(model, X_batch, Y_batch):
    m, v = model.predict_y(X_batch, S)
    S_lik = np.sum(norm.logpdf(Y_batch*Y_std, loc=m*Y_std, scale=Y_std*v**0.5), 2)
    lik = logsumexp(S_lik, 0, b=1/float(S))
    
    mean = np.average(m, 0)
    sq_diff = ((mean - Y_batch)**2)
    return lik, sq_diff

In [26]:
lik, rmse = batch_assess(m_dgp2, Xs, Ys)
print("Log-Likelihood: {} RMSE: {}".format(lik, rmse))

Log-Likelihood: -2.2046342204322613 RMSE: 0.27152198551870377


In [ ]:
m, v = m_dgp2.predict_y(Xs, S)      

In [27]:
# Likelihood con mi metodo
log_num_samples = np.log(S)  
logpdf = norm.logpdf(Ys*Y_std, loc=m*Y_std, scale=Y_std*v**0.5)    
print(np.mean(logsumexp(logpdf, 0) - log_num_samples, 0))

[-2.20266375]


In [10]:
y_unnormalized = Ys * Y_std + Y_mean

In [28]:
# Likelihood con mi metodo
log_num_samples = np.log(S)   
logpdf = norm.logpdf(y_unnormalized, loc=m*Y_std+Y_mean, scale=Y_std*v**0.5)    
print(np.mean(logsumexp(logpdf, 0) - log_num_samples, 0))

[-2.20266375]


In [12]:
# RMSE EXACTO
m, v = m_dgp2.predict_y(Xs, S)
pred = np.mean(m, 0)
rmse = np.sqrt(np.mean((pred - (y_unnormalized - Y_mean)/Y_std)**2))
print(rmse)

0.27085827732633144


In [13]:
from sklearn.metrics import mean_squared_error

mean_squared_error((y_unnormalized - Y_mean)/Y_std, pred)**0.5

0.27085827732633144